## Install Needed Library

In [1]:
# ! pip install mysql-connector-python --quiet
# ! pip install pandas --quiet
# ! pip install SQLAlchemy --quiet

## Importing the Library

In [2]:
import mysql.connector
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError


## Import Credentials

In [ ]:
## Global Variables

host       = "localhost"
user       = "newuser"
password   = "12345"
database   = "db_public_library_project"
table_name = "library_database"

csv_path   = '/Users/moneebhussan/Downloads/LibraryDatasets.csv'
# csv_path = r'B:\Documents\Repository\Xccelerate_Files\Moneeb\Public-Library-Management-System-main\LibraryDatasets.csv'

## Library System Function

In [4]:
# Initialize Database
def database_init(h, u, p, db=None):
    """
    If db is None, returns a connection without selecting a database.
    Otherwise connects to the specified database.
    """
    if db:
        return mysql.connector.connect(host=h, user=u, password=p, database=db)
    else:
        return mysql.connector.connect(host=h, user=u, password=p)

In [5]:
# ## Checker
# database_init(host, user, password)

In [6]:
## Crrate a datbase if not yet existed
def ensure_database_exists():
    """Create the database if it doesn't already exist."""
    conn = database_init(host, user, password)
    cursor = conn.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS `{database}`")
    conn.commit()
    cursor.close()
    conn.close()

In [7]:
## Checker
# ensure_database_exists()

In [8]:
## Create a table and insert bulk data

def table_creation_and_insert():
    """Read the CSV and bulk-insert into MySQL using pandas.to_sql."""
    # 1) Make sure the database exists
    ensure_database_exists()

    # 2) Read your CSV into pandas
    df_books = pd.read_csv(csv_path)

    ## Added to clean columns that have spaces
    df_books.columns = df_books.columns.str.replace(' ', '', regex=False)

    df_books['available'] = df_books['copies']

    ## Added to convert ISBN to string in all times
    try:
        df_books['ISBN'] = df_books['ISBN'].astype('int64').astype(str)
    except:
        pass

    # 3) Build the SQLAlchemy engine (no port in URI)
    engine = create_engine(
        f"mysql+mysqlconnector://{user}:{password}@{host}/{database}",
        echo=False
    )

    try:
        # 4) Write to SQL, replacing the table if it exists
        df_books.to_sql(
            name=table_name,
            con=engine,
            if_exists="fail",  # or 'append'
            index=False,
            chunksize=5
        )
        print("✅ Data inserted successfully!")
    except SQLAlchemyError as e:
        print("❌ Failed to insert data:", e)
    finally:
        # 5) Clean up
        engine.dispose()


In [9]:
# ## Checker
# table_creation_and_insert()

In [10]:
def list_books():

    conn = database_init(host, user, password, database)

    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name}")
    headers = [col[0] for col in cursor.description]
    books = cursor.fetchall()
    print("\nAvailable Books:")

    print(headers)
    for b in books:
        print(b)

    cursor.close()
    conn.close()

In [11]:
# ## Checker
# list_books()

In [12]:
def search_books(keyword):

    conn = database_init(host, user, password, database)

    cursor = conn.cursor()
    query = f"SELECT * FROM {table_name} WHERE Book_title LIKE %s OR Author LIKE %s"
    param = f"%{keyword}%"
    cursor.execute(query, (param, param))
    results = cursor.fetchall()
    headers = [col[0] for col in cursor.description]
    if results:
        print(f"\nSearch Results for '{keyword}':")
        print(headers)
        for r in results:
            print(r)
    else:
        print(f"No books found matching '{keyword}'.")

    cursor.close()
    conn.close()

In [13]:
# ## Checker
# search_books("A Game of Thrones")

In [14]:
def issue_book(ISBN):
    ISBN = str(ISBN)

    conn = database_init(host, user, password, database)
    cursor = conn.cursor()
    
    try:
        # 1) Check how many available copies exist
        query = f"SELECT available FROM {table_name} WHERE ISBN = %s"
        print(f"Executing query: {query} with parameter: {ISBN}")
        cursor.execute(query, (ISBN,))
        row = cursor.fetchone()
    
        if row and row[0] > 0:
            # 2) Decrement available count
            update_query = (
                f"UPDATE {table_name} "
                f"SET available = available - 1 "
                f"WHERE ISBN = %s"
            )
            print(f"Executing update query: {update_query} with parameter: {ISBN}")
            cursor.execute(update_query, (ISBN,))
            conn.commit()
            print("✅ Book issued successfully.")
        else:
            print("⚠️ Book is not available for issue.")
    
    except Exception as e:
        print(f"❌ Database Error: {e}")
    
    finally:
        cursor.close()
        conn.close()


In [15]:
def return_book(ISBN):

    ISBN = str(ISBN)

    conn = database_init(host, user, password, database)
    cursor = conn.cursor()

    try:
        # Fetch the current copies for the given ISBN
        cursor.execute(
            f"SELECT available FROM {table_name} WHERE `ISBN` = %s",
            (ISBN,)
        )
        row = cursor.fetchone()

        if row:
            current_available = row[0]
            # Increment the quantity
            cursor.execute(
                f"UPDATE {table_name} SET available = available + 1 WHERE `ISBN` = %s",
                (ISBN,)
            )
            conn.commit()
            print("Item returned successfully.")
        else:
            print("⚠️ ISBN.")

    except Exception as e:
        print(f"Error: {e}")

    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

In [19]:
def add_book():
    title = input("Enter book title: ")
    author = input("Enter book author: ")
    isbn = input("Enter book ISBN: ")
    try:
        copies = int(input("Enter number of copies: "))
        available = int(input("Enter number of available items: "))
    except ValueError:
        print("Invalid number. Setting copies to 1.")
        copies = 1

    conn = database_init(host, user, password, database)

    cursor = conn.cursor()

    try:
        cursor.execute(
            f"INSERT INTO {table_name} (Book_title, author, isbn, copies, available) VALUES (%s, %s, %s, %s, %s)",
            (title, author, isbn, copies, copies)     
        )
        conn.commit()
        print("Book added successfully.")
    except mysql.connector.IntegrityError as e:
        print(f"⚠️ Failed to add book: {e}")
        if "Duplicate entry" in str(e):
            print("⚠️ ISBN or author name already exists in the database.")

    except mysql.connector.Error as err:
        print(f"Database Error: {err}")

    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()



## Combine All Function for the Project

In [17]:
def main():
    try:
        table_creation_and_insert()
    except:
        print("Table is already created - If you want to recreate then delete the table first")

    while True:
        print("\nLibrary Management System")
        print("1. List all books")
        print("2. Search books")
        print("3. Issue a book")
        print("4. Return a book")
        print("5. Add a new book")
        print("6. Exit")
        choice = input("Enter choice: ")

        if choice == '1':
            list_books()
        elif choice == '2':
            kw = input("Enter title or author to search: ")
            search_books(kw)
        elif choice == '3':
            bid = input("Enter Book ISBN to issue: ")
            issue_book(int(bid))
        elif choice == '4':
            bid = input("Enter Book ISBN to return: ")
            return_book(int(bid))
        elif choice == '5':
            add_book()
        elif choice == '6':
            print("Goodbye!")
            break
        else:
            print("Invalid choice. Please try again.")

In [20]:
main()

Table is already created - If you want to recreate then delete the table first

Library Management System
1. List all books
2. Search books
3. Issue a book
4. Return a book
5. Add a new book
6. Exit
Book added successfully.

Library Management System
1. List all books
2. Search books
3. Issue a book
4. Return a book
5. Add a new book
6. Exit

Available Books:
['Book_title', 'Author', 'ISBN', 'copies', 'available']
('A Game of Thrones', 'George R.R. Martin', '9780100000000', 3, 3)
('The Histories', 'Herodotus', '9780140000000', 5, 5)
('One Hundred Years of Solitude', 'Gabriel Garcia Marquez', '9780140000000', 6, 6)
('Jane Eyre', 'Charlotte Bronte', '9780140000000', 7, 7)
('The Guns of August', 'Barbara W. Tuchman', '9780190000000', 7, 7)
('The Alchemist', 'Paulo Coelho', '9780310000000', 5, 5)
('Sapiens: A Brief History of Humankind', 'Yuval Noah Harari', '9871280000000', 2, 2)
('Animal Farm', 'George Orwell', '9780330000000', 4, 4)
('The Diary of a Young Girl', 'Anne Frank', '978038000